Описание: 
 Разработайте систему для управления сотрудниками в
организации с использованием паттернов Итератор и 

Посредник.


Часть 1: ИтератоA
*2 Создайте класс Employee с аттрибутами: имя, должность,
отдел2
a2 Создайте класс Department, который хранит список
сотрудников и возвращает итератор для перебора
сотрудников2
2 Реализуйте итератор EmployeeIterator для перебора
сотрудников.


Часть 2: Посредник 
 1. Создайте интерфейс Mediator для обмена сообщениями
между сотрудниками разных отделов. 
 2. Реализуйте класс DepartmentMediator для отправки
сообщений между отделами.


Часть 3: Использование Итератора и Посредник4
 Создайте сотрудников и распределите их по отделам2
 Переберите сотрудников отдела с помощью итератора2
 Используйте посредника для отправки сообщений между
сотрудниками разных отделов.


Цель: 
 Показать использование паттернов Итератор и Посредник для
управления сотрудниками и их взаимодействием.

In [5]:
from abc import ABC, abstractmethod
from typing import List, Dict
from datetime import datetime

# Реализация Итератора
class Сотрудник:
    def __init__(self, имя: str, должность: str, отдел: str):
        self.имя = имя
        self.должность = должность
        self.отдел = отдел
    
    def __str__(self):
        return f"{self.имя} ({self.должность}, {self.отдел})"

class ИтераторСотрудников:
    def __init__(self, сотрудники: List[Сотрудник]):
        self._сотрудники = сотрудники
        self._индекс = 0
    
    def __iter__(self):
        return self
    
    def __next__(self) -> Сотрудник:
        if self._индекс < len(self._сотрудники):
            сотрудник = self._сотрудники[self._индекс]
            self._индекс += 1
            return сотрудник
        raise StopIteration

class Отдел:
    def __init__(self, название: str):
        self.название = название
        self._сотрудники: List[Сотрудник] = []
    
    def добавить_сотрудника(self, сотрудник: Сотрудник) -> None:
        if сотрудник.отдел != self.название:
            raise ValueError(f"Отдел сотрудника '{сотрудник.отдел}' не соответствует отделу '{self.название}'")
        self._сотрудники.append(сотрудник)
    
    def удалить_сотрудника(self, сотрудник: Сотрудник) -> None:
        self._сотрудники.remove(сотрудник)
    
    def получить_сотрудников(self) -> List[Сотрудник]:
        return self._сотрудники.copy()
    
    def __iter__(self) -> ИтераторСотрудников:
        return ИтераторСотрудников(self._сотрудники)
    
    def __str__(self) -> str:
        return f"Отдел {self.название} ({len(self._сотрудники)} сотрудников)"
    
    def __contains__(self, сотрудник: Сотрудник) -> bool:
        return сотрудник in self._сотрудники

# Реализация Посредника
class Посредник(ABC):
    @abstractmethod
    def зарегистрировать_отдел(self, отдел: Отдел) -> None:
        pass
    
    @abstractmethod
    def отправить_сообщение(self, отправитель: Сотрудник, сообщение: str, отдел_получателя: str) -> None:
        pass

class ПосредникОтделов(Посредник):
    def __init__(self):
        self._отделы: Dict[str, Отдел] = {}
        self.журнал = []
    
    def зарегистрировать_отдел(self, отдел: Отдел) -> None:
        self._отделы[отдел.название] = отдел
    
    def отправить_сообщение(self, отправитель: Сотрудник, сообщение: str, отдел_получателя: str) -> None:
        if отправитель.отдел not in self._отделы:
            raise ValueError(f"Отдел отправителя '{отправитель.отдел}' не зарегистрирован")
        
        if отдел_получателя not in self._отделы:
            print(f"Отдел '{отдел_получателя}' не найден!")
            return
        
        целевой_отдел = self._отделы[отдел_получателя]
        if отправитель not in self._отделы[отправитель.отдел]:
            raise ValueError(f"Отправитель {отправитель.имя} не является сотрудником отдела '{отправитель.отдел}'")
        
        # Логирование сообщения
        запись = {
            'время': datetime.now(),
            'отправитель': отправитель.имя,
            'отдел_получателя': отдел_получателя,
            'сообщение': сообщение
        }
        self.журнал.append(запись)
        
        print(f"\nСообщение от {отправитель} в {отдел_получателя}:")
        print(f"Текст: {сообщение}")
        print("Получатели:")
        
        есть_получатели = False
        for сотрудник in целевой_отдел:
            if сотрудник != отправитель:
                print(f" - {сотрудник}")
                есть_получатели = True
        
        if not есть_получатели:
            print(" - Нет получателей в целевом отделе (кроме отправителя)")

# Основной код
def main():
    # Создаем отделы
    отделы = {
        "IT": Отдел("IT"),
        "HR": Отдел("HR"),
        "Продажи": Отдел("Продажи")
    }

    # Создаем сотрудников
    сотрудники = [
        Сотрудник("Иван Петров", "Разработчик", "IT"),
        Сотрудник("Елена Смирнова", "Старший разработчик", "IT"),
        Сотрудник("Анна Иванова", "Менеджер HR", "HR"),
        Сотрудник("Борис Козлов", "Рекрутер", "HR"),
        Сотрудник("Сергей Васильев", "Менеджер продаж", "Продажи"),
        Сотрудник("Ольга Николаева", "Представитель продаж", "Продажи")
    ]

    # Распределяем сотрудников по отделам
    for сотрудник in сотрудники:
        try:
            отделы[сотрудник.отдел].добавить_сотрудника(сотрудник)
        except KeyError:
            print(f"Внимание: Отдел '{сотрудник.отдел}' не найден для сотрудника {сотрудник.имя}")
        except ValueError as e:
            print(f"Ошибка добавления сотрудника: {e}")

    # Демонстрация работы итератора
    print("=== Сотрудники по отделам ===")
    for название_отдела, отдел in отделы.items():
        print(f"\n{отдел}:")
        for сотрудник in отдел:
            print(f" - {сотрудник}")

    # Настраиваем посредника
    посредник = ПосредникОтделов()
    for отдел in отделы.values():
        посредник.зарегистрировать_отдел(отдел)

    # Отправка сообщений через посредника
    print("\n=== Обмен сообщениями ===")
    try:
        посредник.отправить_сообщение(сотрудники[0], "Нужно обсудить требования к новому ПО", "HR")
        посредник.отправить_сообщение(сотрудники[2], "Пожалуйста, предоставьте отчеты по продажам за 2 квартал", "Продажи")
        посредник.отправить_сообщение(сотрудники[2], "Это сообщение не будет доставлено", "Маркетинг")
        
        # Тест с несуществующим сотрудником
        несуществующий_сотрудник = Сотрудник("Неизвестный", "Стажер", "IT")
        посредник.отправить_сообщение(несуществующий_сотрудник, "Тестовое сообщение", "HR")
    except ValueError as e:
        print(f"\nОшибка: {e}")

    # Вывод журнала сообщений
    print("\n=== Журнал сообщений ===")
    for запись in посредник.журнал:
        print(f"{запись['время']}: {запись['отправитель']} -> {запись['отдел_получателя']}: {запись['сообщение']}")

if __name__ == "__main__":
    main()

=== Сотрудники по отделам ===

Отдел IT (2 сотрудников):
 - Иван Петров (Разработчик, IT)
 - Елена Смирнова (Старший разработчик, IT)

Отдел HR (2 сотрудников):
 - Анна Иванова (Менеджер HR, HR)
 - Борис Козлов (Рекрутер, HR)

Отдел Продажи (2 сотрудников):
 - Сергей Васильев (Менеджер продаж, Продажи)
 - Ольга Николаева (Представитель продаж, Продажи)

=== Обмен сообщениями ===

Сообщение от Иван Петров (Разработчик, IT) в HR:
Текст: Нужно обсудить требования к новому ПО
Получатели:
 - Анна Иванова (Менеджер HR, HR)
 - Борис Козлов (Рекрутер, HR)

Сообщение от Анна Иванова (Менеджер HR, HR) в Продажи:
Текст: Пожалуйста, предоставьте отчеты по продажам за 2 квартал
Получатели:
 - Сергей Васильев (Менеджер продаж, Продажи)
 - Ольга Николаева (Представитель продаж, Продажи)
Отдел 'Маркетинг' не найден!

Ошибка: Отправитель Неизвестный не является сотрудником отдела 'IT'

=== Журнал сообщений ===
2025-04-26 07:43:10.449842: Иван Петров -> HR: Нужно обсудить требования к новому ПО
2025-04-